In [ ]:
Добавить в index_reverse статистики, чтобы быстрее работало.

In [8]:
import re
import nltk
from nltk.tokenize import word_tokenize
import string
import pymorphy2
import stop_words
from stop_words import get_stop_words
from IPython.display import HTML, display
import numpy as np
import pickle

In [2]:
def list_of_news():
    collection = [()]*1000
    for i in range(0, 1000):
        f = open('News_corpus/article{}.txt'.format(i), encoding='utf8')
        f1 = f.read()
        f.close()
        link_text = re.findall('@url (.+?)\n(.+)', f1)
        title = re.findall('@ti (.+?)\n', f1)[0]
        if link_text != []:
            text = link_text[0][1]
            text = re.sub('&.+?;', '', text)
            link = link_text[0][0]
        collection[i] = (link, text, title)
    return collection

In [3]:
collection_strings = list_of_news()

In [51]:
with open('collection_with_data.pickle', 'wb') as f:
    pickle.dump(collection_strings, f)

In [4]:
collection_strings[:3]

[('http://inmozhaisk.ru/novosti/obschestvo/goryachaya-liniya-po-voprosam-osennego-prizyva-proydyot-v-mozhayskom-rayone-17-oktyabrya',
  '17 октября с 10.00 до 11.00 военный комиссар городов Можайск и Руза, Можайского и Рузского районов Московской области проведёт Горячую линию для жителей Можайского и Рузского муниципальных районов по вопросам осеннего призыва граждан на военную службу.На все интересующие вопросы по осеннему призыву вам ответит военный комиссар городов Можайск и Руза, Можайского и Рузского районов Московской области Меладзе Руслан Бесхведович и другие должностные лица военного комиссариата по тел.: 8 (49638) 20-990.',
  '«Горячая линия» по вопросам осеннего призыва пройдёт в Можайском районе 17 октября'),
 ('http://inmozhaisk.ru/novosti/obschestvo/mozhayskie-pozharnye-i-spasateli-otmetili-10-letie-so-dnya-obrazovaniya-gku-mo-mosoblpozhspas',
  'Поздравить спасателей Можайского муниципального района и Рузского городского округа пришли главы городских и сельских поселени

In [5]:
def delete_stop_words(s, lng):
    stop_words = get_stop_words(lng)
    s -= set(stop_words)
    return s

morph = pymorphy2.MorphAnalyzer()
def preprocessing(text):
    text = ''.join([i for i in text if i not in set(string.punctuation)])
    text = text.lower()
    words = word_tokenize(text)
    lemmas = set()
    for word in words:
        lemmas.add(morph.parse(word)[0].normal_form)
    lemmas_new = delete_stop_words(lemmas, 'ru')
    return lemmas_new

In [6]:
collection = []
for triple in collection_strings:
    collection.append(preprocessing(triple[1]))

In [9]:
with open('collection.pickle', 'wb') as f:
    pickle.dump(collection, f)

In [10]:
collection[10]

{'17',
 '2000',
 '500',
 '540',
 'английский',
 'благодарственный',
 'владеть',
 'воспитание',
 'вручить',
 'выигрывать',
 'высокий',
 'глава',
 'город',
 'городашкола',
 'грамота',
 'гусев',
 'дать',
 'детейсейчас',
 'деятельность',
 'директор',
 'заведение',
 'иностранный',
 'культура',
 'министерство',
 'многие',
 'многий',
 'можаец',
 'москва',
 'наградить',
 'наталья',
 'немецкий',
 'номинация',
 'обучаться',
 'обучение',
 'окончить',
 'олимп',
 'открытый',
 'отлично',
 'отличный',
 'письмо',
 'подмосковье',
 'преподавательский',
 'произношение',
 'профессионализм',
 'прошедшее',
 'путёвка',
 'работа',
 'родственный',
 'среди',
 'учебный',
 'ученик',
 'французский',
 'хороший',
 'школа',
 'язык'}

In [22]:
def index_reverse(collection):
    dic = {}
    for i in range(0, len(collection)):
        s = collection[i]
        for j in s:
            if j in dic:
                dic[j].append(i)
            else:
                dic[j] = [i]
    return dic

In [23]:
def average_len(D):
    return np.mean([len(i) for i in D])

In [24]:
from math import log

k1 = 1.2
k2 = 100
b = 0.75
R = 0.0

from math import log

k1 = 2.0
b = 0.75

def score_BM25_i(n, fq, N, dl, avdl):
    K = compute_K(dl, avdl)
    IDF = log((N - n + 0.5) / (n + 0.5))
    frac = ((k1 + 1) * fq) / (K + fq)
    return IDF * frac


def compute_K(dl, avdl):
    return k1 * ((1-b) + b * (float(dl)/float(avdl)))

def score_BM25(Q, D, D_j, i_reverse, avdl):
    s = 0
    for i in Q:
        f = 0
        if i in i_reverse:
            f = len([j==i for j in D_j])
            s += score_BM25_i(len(i_reverse[i]), f, len(D), len(D_j), average_len(collection))
    return s

In [25]:
ir = index_reverse(collection)

In [44]:
with open('ir.pickle', 'wb') as f:
    pickle.dump(ir, f)

In [ ]:
with open('collection.pickle', 'rb') as f:
    D = pickle.load(f)

with open('ir.pickle', 'rb') as f:
    ir = pickle.load(f)

with open('collection_with_data.pickle', 'rb') as f:
    collection_with_data = pickle.load(f)

In [66]:
def search(Q):
    if type(Q) != str:
        raise BaseException('Request must be str type')
    if re.compile("\w+( \w+)*").match(Q).span() != (0, len(Q)):
        raise BaseException('Input words seperated by spaces')
    
    Q = Q.lower()
    Q_words = word_tokenize(Q)
    morph = pymorphy2.MorphAnalyzer()
    Q_lemmas = set()
    for word in Q_words:
        Q_lemmas.add(morph.parse(word)[0].normal_form)
    Q_lemmas = delete_stop_words(Q_lemmas, 'ru')
        
    good_Ds = []
    for lemma in Q_lemmas:
        if lemma in ir:
            good_Ds += ir[lemma]
    good_Ds = set(good_Ds)
#    print(len(good_Ds))
#    print(good_Ds)
    scores = {}
    for i in good_Ds:
        d = collection[i]
        link = collection_with_data[i][0]
        head = collection_with_data[i][2]
#        print(d)
        scores[(link, head)] = score_BM25(Q_lemmas, D, d, ir, average_len(D))
#    sorted_answer = sorted(scores, key=scores.get)
#    for i in sorted_answer[::-1]:
#        link = 'http://google.ru/' + '<a href="' + collection_strings[i][0] + '">' + collection_strings[i][2] + '</a>'
#        link = '{} — {} — {}'.format(i, collection_strings[i][0], scores[i])
#        display(HTML(str(link)))
    return sorted(scores, key=scores.get, reverse=True)[:10]

In [67]:
answer = search('каникулы на новый год и рождество')
answer

[('http://inmozhaisk.ru/novosti/guberniya/gubernator-podmoskovya-andrey-vorobev-rasskazal-ob-itogah-aprelya-v-pryamom-efire-telekanala-360deg-podmoskove',
  'Губернатор Подмосковья Андрей Воробьев рассказал об итогах апреля в прямом эфире телеканала «360° Подмосковье»'),
 ('http://inmozhaisk.ru/novosti/guberniya/gubernator-podmoskovya-andrey-vorobev-rasskazal-ob-itogah-aprelya-v-pryamom-efire-telekanala-360deg-podmoskove',
  'В Можайском районе формируется &quot;Бессмертный полк&quot;'),
 ('http://inmozhaisk.ru/novosti/guberniya/gubernator-podmoskovya-andrey-vorobev-rasskazal-ob-itogah-aprelya-v-pryamom-efire-telekanala-360deg-podmoskove',
  'Всероссийские акции &quot;Фронтовой хлеб&quot; и &quot;Георгиевская ленточка&quot; состоятся в Можайске'),
 ('http://inmozhaisk.ru/novosti/guberniya/gubernator-mo-andrey-vorobev-podvel-itogi-mesyaca-v-efire-telekanala-360deg-podmoskove',
  'Губернатор МО Андрей Воробьев подвел итоги месяца в эфире телеканала «360° Подмосковье»'),
 ('http://inmozha

In [70]:
for i in answer[:10]:
    display(HTML('<a href="{}">{}</a>'.format(i[0],i[1])))